## Import librarys

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(style='white', context='notebook', palette='deep')

## Load data

In [2]:
df = pd.read_csv("./input/2018.csv")

c:\projects\python\autotel-ml\venv\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,מספר רכב,מספר הזמנה,קטגוריית שריון,ממוצע מרחק לנסיעה,דקות לחיוב,כתובת התחלה,כתובת סיום,תאריך ושעה התחלת שריון,תאריך ושעה סיום הזמנה
0,46-525-81,60386,שריון חינמי,8,13,"גורדון 86-88, תל אביב יפו, ישראל","הרב רובינשטיין 61, תל אביב יפו, ישראל",23/10/2017 0:08,23/10/2017 0:31
1,48-437-81,60387,שריון חינמי,3,8,"יעקב אפטר 3, תל אביב יפו, ישראל","ז'בוטינסקי 10, תל אביב יפו, ישראל",23/10/2017 0:09,23/10/2017 0:21
2,63-887-81,60388,שריון חינמי,0,2,"רופין 32-34, תל אביב יפו, ישראל","רופין 32-34, תל אביב יפו, ישראל",23/10/2017 0:10,23/10/2017 0:15
3,63-906-81,60389,שריון חינמי,4,9,"שדרות מסריק 19, תל אביב יפו, ישראל","אונקלוס 3, תל אביב יפו, ישראל",23/10/2017 0:12,23/10/2017 0:23
4,60-589-55,60390,שריון חינמי,2,6,"הירקון 188, תל אביב יפו, ישראל","גורדון 83-85, תל אביב יפו, ישראל",23/10/2017 0:12,23/10/2017 0:21


In [4]:
df = df.rename({'מספר רכב': 'Car id', 'מספר הזמנה': 'Order number','קטגוריית שריון':'Category',
                 'ממוצע מרחק לנסיעה':'Avg distance','דקות לחיוב':'Billing minutes','כתובת התחלה':'Start Address',
                 'כתובת סיום':'End Address','תאריך ושעה התחלת שריון':'Start time','תאריך ושעה סיום הזמנה':'End time'},
                axis=1) 

In [5]:
df.head()

,Car id,Order number,Category,Avg distance,Billing minutes,Start Address,End Address,Start time,End time
0,46-525-81,60386,שריון חינמי,8,13,"גורדון 86-88, תל אביב יפו, ישראל","הרב רובינשטיין 61, תל אביב יפו, ישראל",23/10/2017 0:08,23/10/2017 0:31
1,48-437-81,60387,שריון חינמי,3,8,"יעקב אפטר 3, תל אביב יפו, ישראל","ז'בוטינסקי 10, תל אביב יפו, ישראל",23/10/2017 0:09,23/10/2017 0:21
2,63-887-81,60388,שריון חינמי,0,2,"רופין 32-34, תל אביב יפו, ישראל","רופין 32-34, תל אביב יפו, ישראל",23/10/2017 0:10,23/10/2017 0:15
3,63-906-81,60389,שריון חינמי,4,9,"שדרות מסריק 19, תל אביב יפו, ישראל","אונקלוס 3, תל אביב יפו, ישראל",23/10/2017 0:12,23/10/2017 0:23
4,60-589-55,60390,שריון חינמי,2,6,"הירקון 188, תל אביב יפו, ישראל","גורדון 83-85, תל אביב יפו, ישראל",23/10/2017 0:12,23/10/2017 0:21


## Data Preperation

### Create new rows from End Address and End time

In [6]:
df_temp = df.copy()

In [7]:
df.drop(["End Address","End time"],axis=1,inplace=True)
df_temp.drop(["Start Address","Start time"],axis=1,inplace=True)

In [8]:
df = df.rename({"Start Address":"Address","Start time": "Time"},axis=1)
df_temp = df_temp.rename({"End Address":"Address","End time": "Time"},axis=1)

In [9]:
df = pd.concat([df,df_temp])

In [10]:
df.sort_values(["Order number","Time"],axis=0,inplace=True)

### check for null and missing values

In [11]:
# Fill empty and NaNs values with NaN
df = df.fillna(np.nan)

In [12]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300004 entries, 0 to 150001
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Car id           300004 non-null  object
 1   Order number     300004 non-null  int64 
 2   Category         300004 non-null  object
 3   Avg distance     300004 non-null  int64 
 4   Billing minutes  275694 non-null  object
 5   Address          286966 non-null  object
 6   Time             286358 non-null  object
dtypes: int64(2), object(5)
memory usage: 12.6+ MB


Car id                 0
Order number           0
Category               0
Avg distance           0
Billing minutes    24310
Address            13038
Time               13646
dtype: int64

### Splitting address to country, city, street

In [13]:
split_col = df["Address"].str.split(",", expand=True)
split_col[:3]

,0,1,2,3
0,גורדון 86-88,תל אביב יפו,ישראל,None
0,הרב רובינשטיין 61,תל אביב יפו,ישראל,None
1,יעקב אפטר 3,תל אביב יפו,ישראל,None


In [14]:
df["Street"] = split_col[0]
df["City"] = split_col[1]
df["Country"] = split_col[2]

#### extracting street without numbers

In [60]:
df["Street_c"] = df['Street'].str.replace(r'(\d+)(-(\d+))*', r'')

#### extract clean address for google api use

In [64]:
df['Address_c'] = df['Address'].str.replace(r'(\d+)-(\d+)', r'\1')
df['Address_c']

0                  גורדון 86, תל אביב יפו, ישראל
0          הרב רובינשטיין 61, תל אביב יפו, ישראל
1                יעקב אפטר 3, תל אביב יפו, ישראל
1              ז'בוטינסקי 10, תל אביב יפו, ישראל
2                   רופין 32, תל אביב יפו, ישראל
                           ...                  
149999          מגן אברהם 16, תל אביב יפו, ישראל
150000                 רבינא, תל אביב יפו, ישראל
150000    שדרות בן גוריון 55, תל אביב יפו, ישראל
150001            נמל יפו 30, תל אביב יפו, ישראל
150001                 צפת 1, תל אביב יפו, ישראל
Name: Address_c, Length: 300004, dtype: object

### Extract date and time

In [15]:
df["Time"] = pd.to_datetime(df["Time"])

In [16]:
# df["Date"] = df["Time"].dt.date
# df["time"] = df["Time"].dt.time
df["Year"] = df["Time"].dt.year
df["Month"] = df["Time"].dt.month
df["Day"] = df["Time"].dt.day
df["Hour"] = df["Time"].dt.hour
df["Minute"] = df["Time"].dt.minute

## Visualization

In [185]:
counts = df['Street_c'].value_counts()
df[df['Street_c'].isin(counts[counts > 1000].index)]

,Car id,Order number,Category,Avg distance,Billing minutes,Address,Time,Street,City,Country,Year,Month,Day,Hour,Minute,Street_c,Address_c
0,46-525-81,60386,שריון חינמי,8,13,"גורדון 86-88, תל אביב יפו, ישראל",2017-10-23 00:08:00,גורדון 86-88,תל אביב יפו,ישראל,2017.0,10.0,23.0,0.0,8.0,גורדון,"גורדון 86, תל אביב יפו, ישראל"
1,48-437-81,60387,שריון חינמי,3,8,"יעקב אפטר 3, תל אביב יפו, ישראל",2017-10-23 00:09:00,יעקב אפטר 3,תל אביב יפו,ישראל,2017.0,10.0,23.0,0.0,9.0,יעקב אפטר,"יעקב אפטר 3, תל אביב יפו, ישראל"
1,48-437-81,60387,שריון חינמי,3,8,"ז'בוטינסקי 10, תל אביב יפו, ישראל",2017-10-23 00:21:00,ז'בוטינסקי 10,תל אביב יפו,ישראל,2017.0,10.0,23.0,0.0,21.0,ז'בוטינסקי,"ז'בוטינסקי 10, תל אביב יפו, ישראל"
3,63-906-81,60389,שריון חינמי,4,9,"שדרות מסריק 19, תל אביב יפו, ישראל",2017-10-23 00:12:00,שדרות מסריק 19,תל אביב יפו,ישראל,2017.0,10.0,23.0,0.0,12.0,שדרות מסריק,"שדרות מסריק 19, תל אביב יפו, ישראל"
4,60-589-55,60390,שריון חינמי,2,6,"הירקון 188, תל אביב יפו, ישראל",2017-10-23 00:12:00,הירקון 188,תל אביב יפו,ישראל,2017.0,10.0,23.0,0.0,12.0,הירקון,"הירקון 188, תל אביב יפו, ישראל"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149996,63-896-81,240593,שריון חינמי,3,10,"ויצמן‬ 56, תל אביב יפו, ישראל",2018-01-05 00:03:00,ויצמן‬ 56,תל אביב יפו,ישראל,2018.0,1.0,5.0,0.0,3.0,ויצמן‬,"ויצמן‬ 56, תל אביב יפו, ישראל"
149996,63-896-81,240593,שריון חינמי,3,10,"צקלג 19, תל אביב יפו, ישראל",2018-04-30 23:48:00,צקלג 19,תל אביב יפו,ישראל,2018.0,4.0,30.0,23.0,48.0,צקלג,"צקלג 19, תל אביב יפו, ישראל"
149998,60-583-55,240595,Canceled,0,0,"ד""ר ג'ורג' וייז 6, תל אביב יפו, ישראל",2018-04-30 23:51:00,"ד""ר ג'ורג' וייז 6",תל אביב יפו,ישראל,2018.0,4.0,30.0,23.0,51.0,"ד""ר ג'ורג' וייז","ד""ר ג'ורג' וייז 6, תל אביב יפו, ישראל"
149999,71-785-55,240596,שריון חינמי,2,7,"פלורנטין 31, תל אביב יפו, ישראל",2018-01-05 00:03:00,פלורנטין 31,תל אביב יפו,ישראל,2018.0,1.0,5.0,0.0,3.0,פלורנטין,"פלורנטין 31, תל אביב יפו, ישראל"
